In [2]:
%pip install calamancy

  Using cached calamanCy-0.2.2-py3-none-any.whl.metadata (15 kB)
  Using cached spacy_transformers-1.3.5-cp313-cp313-win_amd64.whl
  Using cached transformers-4.36.2-py3-none-any.whl.metadata (126 kB)
  Using cached tokenizers-0.15.2-cp313-cp313-win_amd64.whl
Using cached calamanCy-0.2.2-py3-none-any.whl (9.8 kB)
Using cached transformers-4.36.2-py3-none-any.whl (8.2 MB)

  Attempting uninstall: tokenizers

    Found existing installation: tokenizers 0.20.3

    Uninstalling tokenizers-0.20.3:

      Successfully uninstalled tokenizers-0.20.3

   ---------------------------------------- 0/4 [tokenizers]
   ---------------------------------------- 0/4 [tokenizers]
  Attempting uninstall: transformers
   ---------------------------------------- 0/4 [tokenizers]
   ---------- ----------------------------- 1/4 [transformers]
    Found existing installation: transformers 4.45.2
   ---------- ----------------------------- 1/4 [transformers]
   ---------- ----------------------------- 1/4 [tr

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gliner 0.2.8 requires transformers>=4.38.2, but you have transformers 4.36.2 which is incompatible.


In [3]:
import spacy, calamancy

print("spaCy:", spacy.__version__)
print("Calamancy:", calamancy.__version__)



spaCy: 3.8.7
Calamancy: 0.2.2


In [ ]:
import random

random_seed = 42
random.seed = 42

In [ ]:
import os

dataset_name = "batch_1-2"
dataset_file_path = f"../assets/json_records/{dataset_name}"



os.makedirs(dataset_name, exist_ok=True)


In [5]:
import json, unicodedata, collections

# Load dataset
with open(f"{dataset_file_path}/records_clean.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Store character info
char_stats = collections.Counter()
char_examples = {}

def analyze_text(text, rec_id):
    for ch in text:
        if ord(ch) > 127 or ch in ["\u200b", "\u00a0"]:  # non-ASCII, zero-width, NBSP
            key = f"{ch} | U+{ord(ch):04X} | {unicodedata.name(ch, 'UNKNOWN')}"
            char_stats[key] += 1
            if key not in char_examples:
                # keep first 2 examples only
                char_examples[key] = []
            if len(char_examples[key]) < 2:
                snippet = text[:100].replace("\n", " ")
                char_examples[key].append({"record_id": rec_id, "sample": snippet})

# Scan all records
for rec in data:
    text = rec["fields"]["Text"]
    analyze_text(text, rec.get("id"))

# Save summary
summary = []
for char_key, count in char_stats.most_common():
    summary.append({
        "char": char_key,
        "count": count,
        "examples": char_examples.get(char_key, [])
    })

with open(f"{dataset_file_path}/weird_chars.json", "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)

print("✅ Weird characters scanned. Results saved to weird_chars.json")


✅ Weird characters scanned. Results saved to weird_chars.json


In [9]:
print("Total records:", len(data))

Total records: 2700


In [96]:
def find_overlaps(entities, text):
    """Detect overlaps in list of (start, end, label) and return span text too."""
    overlaps = []
    # Sort by start index
    entities_sorted = sorted(entities, key=lambda x: (x[0], x[1]))
    for i in range(len(entities_sorted) - 1):
        start1, end1, label1 = entities_sorted[i]
        start2, end2, label2 = entities_sorted[i+1]
        # If the next entity starts before the current one ends → overlap
        if start2 < end1:
            overlaps.append({
                "first": {
                    "start": start1,
                    "end": end1,
                    "label": label1,
                    "text": text[start1:end1]
                },
                "second": {
                    "start": start2,
                    "end": end2,
                    "label": label2,
                    "text": text[start2:end2]
                }
            })
    return overlaps

# ---------- Helper functions -------------
def remove_white_space_from_span(text : str, start : int, end : int):
    curr_text_len = len(text)
    adjusted = False
    # print(text)

    text = text.lstrip()
    if len(text) < curr_text_len:
        end -= curr_text_len - len(text)
        curr_text_len = len(text)
        adjusted = True

    text = text.rstrip()
    if len(text) < curr_text_len:
        end -= curr_text_len - len(text)
        curr_text_len = len(text)
        adjusted = True
    
    if adjusted:
        out = {
            "text": text,
            "start": start,
            "end": end
        }
        display(f"Cleaned: {out}")
        return out
    else:
        return None

In [97]:
sample = {"text": "Ayon sa Dangadang (Sandatahang Pakikibaka), pahayagang masa sa rehiyon, sa isyu noong Nobyembre, una'y nagpetisyon ang mga taumbaryo sa meyor na pahintuin ang mga manggugubat (forester) ng BFD na magtanim sa kanilang mga lupaing sinasaka. Nang di ito pinakinggan, pinagbunot ng galit na mga mamamayang Igorot ang nakatanim na mga punong-kahoy.",
"span_text": "Dangadang ",
"start": 8,
"end": 18,}

remove_white_space_from_span(sample["span_text"], sample["start"], sample["end"])

"Cleaned: {'text': 'Dangadang', 'start': 8, 'end': 17}"

{'text': 'Dangadang', 'start': 8, 'end': 17}

In [ ]:
from sklearn.model_selection import train_test_split

stratified_sample, _ = train_test_split(data, test_size=0.999, stratify=data[[]])

In [ ]:
import json
import spacy
import calamancy
import unicodedata
from spacy.util import filter_spans

# Load calamancy Tagalog model
nlp = calamancy.load("tl_calamancy_md-0.2.0")

# Load JSON
# with open(f"{dataset_file_path}/records.json", "r", encoding="utf-8") as f:
#     data = json.load(f)

# Shuffle dataset for random split
# todo: functional stratified sampling
random.shuffle(data)

# Split ratios
train_ratio, dev_ratio, test_ratio = 0.7, 0.1, 0.2
n = len(data)
n_train = int(n * train_ratio)
n_dev = int(n * dev_ratio)

train_data = data[:n_train]
dev_data = data[n_train:n_train+n_dev]
test_data = data[n_train+n_dev:]

# Collect misaligned spans for logging
misaligned_log = []
overlap_log = []
dropped_log = []
neg_sample_log = []

    
# ---------- Helper: spaCy format ----------
def to_spacy_format(records):
    spacy_data = []
    for idx, record in enumerate(records):
        text = unicodedata.normalize("NFC", record["fields"]["Text"])
        entities = []
        for resp in record.get("responses", {}).get("entity_type", []):
            for ent in resp.get("value", []):
                entities.append((ent["start"], ent["end"], ent["label"]))

        # --- check overlaps here ---
        overlaps = find_overlaps(entities, text)
        if overlaps:
            overlap_log.append({
                "record_id": record.get("id"),
                "text": text,
                "overlaps": overlaps
            })
            dropped_log.append({
                "record_id": record.get("id"),
                "text": text,
                "reason": "overlap",
                "details": overlaps
            })
            continue  # still skip the record completely

        # --- continue with alignment check ---
        aligned_entities = []
        for start, end, label in entities:
            # doc = nlp.make_doc(text)
            # span = doc.char_span(start, end, label=label)
            cleaned = remove_white_space_from_span(text[start:end], start, end)

            if (cleaned is not None):
                # aligned_entities.append((cleaned["start"], cleaned["end"], label))
                misaligned_log.append({
                    "record_id": record.get("id"),
                    "text": text,
                    "span_text": text[start:end],
                    "start": start,
                    "end": end,
                    "label": label
                })
                start = cleaned["start"]
                end = cleaned["end"]
                # span_text = cleaned["text"]
                if (start == end):
                    # empty span
                    dropped_log.append({
                        "record_id": record.get("id"),
                        "text": text,
                        "reason": "overlap",
                        "details": overlaps
                    })
                    continue
            aligned_entities.append((start, end, label))

        # --- final decision ---
        if not aligned_entities:
            #  keep as negative example, but log it
            spacy_data.append([text, {"entities": []}])
            neg_sample_log.append({
                "record_id": record.get("id"),
                "text": text,
                "reason": "no entities (kept as negative example)"
            }) # NO NEED TO LOG AS DROPPED SINCE THEY ARE NEEDED AS NEGATIVE SAMPLES
        else:
            spacy_data.append([text, {"entities": aligned_entities}])
    return spacy_data


# ---------- Helper: BIO format ----------
def to_bio_format(records):
    bio_sents = []
    for record in records:
        text = unicodedata.normalize("NFC", record["fields"]["Text"])
        entities = []
        for resp in record.get("responses", {}).get("entity_type", []):
            for ent in resp.get("value", []):
                entities.append((ent["start"], ent["end"], ent["label"]))
        
        doc = nlp(text)
        tags = ["O"] * len(doc)

        for start, end, label in entities:
            for i, token in enumerate(doc):
                if token.idx >= start and token.idx < end:
                    if token.idx == start:
                        tags[i] = f"B-{label}"
                    else:
                        tags[i] = f"I-{label}"

        sent_tags = [(token.text, tags[i]) for i, token in enumerate(doc)]
        bio_sents.append(sent_tags)
    return bio_sents

In [100]:
import os

corpus_path = f"../assets/corpus/{dataset_name}"

os.makedirs(f"{corpus_path}/json", exist_ok=True)
os.makedirs(f"{corpus_path}/bio", exist_ok=True)

# ---------- Save outputs ----------
for split_name, split_data in [("train", train_data), ("dev", dev_data), ("test", test_data)]:
    # spaCy JSON → inside /json folder
    spacy_data = to_spacy_format(split_data)
    with open(f"{corpus_path}/json/{split_name}.json", "w", encoding="utf-8") as f:
        json.dump(spacy_data, f, ensure_ascii=False, indent=2)

    # BIO → inside /bio folder
    bio_data = to_bio_format(split_data)
    with open(f"{corpus_path}/bio/{split_name}.bio", "w", encoding="utf-8") as f:
        for sent in bio_data:
            for token, tag in sent:
                f.write(f"{token}\t{tag}\n")
            f.write("\n")

# Save misaligned spans log in root
with open(f"{corpus_path}/misaligned.json", "w", encoding="utf-8") as f:
    json.dump(misaligned_log, f, ensure_ascii=False, indent=2)

print("✅ Saved: train/dev/test in both spaCy JSON (/json) and BIO (/bio) formats")
print(f"⚠️ Misaligned spans logged: {len(misaligned_log)}")

with open(f"{corpus_path}/overlaps.json", "w", encoding="utf-8") as f:
    json.dump(overlap_log, f, ensure_ascii=False, indent=2)

print(f"⚠️ Overlapping spans logged: {len(overlap_log)}")

# ---------- Save dropped log ----------
with open(f"{corpus_path}/dropped.json", "w", encoding="utf-8") as f:
    json.dump(dropped_log, f, ensure_ascii=False, indent=2)

print(f"❌ Dropped entries logged: {len(dropped_log)}")

"Cleaned: {'text': 'US', 'start': 150, 'end': 152}"

"Cleaned: {'text': 'Dangadang', 'start': 8, 'end': 17}"

"Cleaned: {'text': 'Demokratikong Republikang Bayan ng Korea', 'start': 556, 'end': 596}"

"Cleaned: {'text': 'Pilipino', 'start': 141, 'end': 149}"

"Cleaned: {'text': 'Okt. 20', 'start': 106, 'end': 113}"

...

"Cleaned: {'text': 'Enero', 'start': 193, 'end': 198}"

"Cleaned: {'text': 'HULYO 1985', 'start': 63, 'end': 73}"

"Cleaned: {'text': 'korporasyong transnasyonal', 'start': 36, 'end': 62}"

"Cleaned: {'text': 'SAMBUTANI', 'start': 54, 'end': 63}"

✅ Saved: train/dev/test in both spaCy JSON (/json) and BIO (/bio) formats
⚠️ Misaligned spans logged: 15
⚠️ Overlapping spans logged: 4
❌ Dropped entries logged: 4


In [101]:
with open(f"{corpus_path}/neg_sample.json", "w", encoding="utf-8") as f:
    json.dump(neg_sample_log, f, ensure_ascii=False, indent=2)

print(f"⚠️ Negative example spans logged: {len(neg_sample_log)}")

⚠️ Negative example spans logged: 464


In [102]:
import json

def count_json_entries(json_file):
    with open(f"{corpus_path}/{json_file}", "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"{json_file}: {len(data)} entries")
    return len(data)

count_json_entries("json/train.json")
count_json_entries("json/dev.json")
count_json_entries("json/test.json")


json/train.json: 1888 entries
json/dev.json: 270 entries
json/test.json: 538 entries


538

In [ ]:
import json, unicodedata

# Load helper
def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def to_key(record):
    # Prefer ID if available
    if isinstance(record, dict):
        if "id" in record:
            return str(record["id"])

        fields = record.get("fields")
        if isinstance(fields, dict):
            return unicodedata.normalize("NFC", fields.get("Text", ""))
        elif isinstance(fields, list):
            # If fields is a list, join as string for uniqueness
            return unicodedata.normalize("NFC", " ".join(map(str, fields)))
    
    # fallback: stringify entire record
    return unicodedata.normalize("NFC", str(record))

# Load all splits
train = load_json("json/train.json")
dev   = load_json("json/dev.json")
test  = load_json("json/test.json")

train_keys = {to_key(r) for r in train}
dev_keys   = {to_key(r) for r in dev}
test_keys  = {to_key(r) for r in test}

# Report sizes
print("Train size:", len(train_keys))
print("Dev size:", len(dev_keys))
print("Test size:", len(test_keys))

# Overlaps
overlap_train_dev = train_keys & dev_keys
overlap_train_test = train_keys & test_keys
overlap_dev_test = dev_keys & test_keys

print("Train ∩ Dev:", len(overlap_train_dev))
print("Train ∩ Test:", len(overlap_train_test))
print("Dev ∩ Test:", len(overlap_dev_test))

# Global uniqueness
all_keys = train_keys | dev_keys | test_keys
total = len(train) + len(dev) + len(test)
print("Total samples:", total)
print("Unique samples:", len(all_keys))
print("Duplicates overall:", total - len(all_keys))

# Optional: save actual duplicates for inspection
duplicates = {
    "train_dev": list(overlap_train_dev),
    "train_test": list(overlap_train_test),
    "dev_test": list(overlap_dev_test),
}
with open("split_duplicates.json", "w", encoding="utf-8") as f:
    json.dump(duplicates, f, ensure_ascii=False, indent=2)

print("✅ Duplicate report saved to split_duplicates.json")


Train size: 837
Dev size: 119
Test size: 238
Train ∩ Dev: 0
Train ∩ Test: 0
Dev ∩ Test: 0
Total samples: 1194
Unique samples: 1194
Duplicates overall: 0
✅ Duplicate report saved to split_duplicates.json


In [ ]:
import json
import spacy
from spacy.tokens import DocBin

os.makedirs("spacy", exist_ok=True)

nlp = calamancy.load("tl_calamancy_md-0.2.0")

def convert(json_file, spacy_file):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    doc_bin = DocBin()
    for text, annot in data:
        # normalize text to avoid encoding mismatches
        text = unicodedata.normalize("NFC", text)
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is None:
                print(f"⚠️ Misaligned span skipped: {text[start:end]}")
            else:
                ents.append(span)

        # filter overlaps and assign
        doc.ents = filter_spans(ents)
        doc_bin.add(doc)

    # save binary .spacy file
    doc_bin.to_disk(spacy_file)
    print(f"Saved {spacy_file}")

# Convert splits
convert("json/train.json", "spacy/train.spacy")
convert("json/dev.json", "spacy/dev.spacy")
convert("json/test.json", "spacy/test.spacy")


Saved spacy/train.spacy
Saved spacy/dev.spacy
Saved spacy/test.spacy
